This notebook has to run in an environment with SAM, on Windows it will require WSL.

In [1]:
from imaris_ims_file_reader.ims import ims
import dask.array as da
import os
import gc
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tifffile import imread
from skimage.color import gray2rgb
import matplotlib.pyplot as plt
from skimage.segmentation import clear_border
from tqdm import tqdm
import pickle as pkl

from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

import torch
torch.cuda.is_available()

True

In [2]:
device = torch.device("cuda")

torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
# turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
if torch.cuda.get_device_properties(0).major >= 8:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

In [3]:
im_path = r'/mnt/d/data_analysis/2025_Birder_mito/88EM87C 25x25_ashlar.ome.tif'
output_dir = r'/mnt/d/data_analysis/2025_Birder_mito/C_00_analysis'
output_sub_dir = 'mitos_sam'
prefix_save = '88EM87C_'

df_path = os.path.join(output_dir,f'{prefix_save}axons.pkl')

axon_res = 3
mitos_res = 0
row_offset = 0 # used if df was created as a test on a smaller image
col_offset = 0 # used if df was created as a test on a smaller image

sam2_checkpoint = "/home/kasia/sam/sam2/checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

os.makedirs(os.path.join(output_dir,output_sub_dir),exist_ok=True)

In [4]:
store = imread(im_path, aszarr=True)
im = da.from_zarr(store,mitos_res)
im

dask.array<from-zarr, shape=(22084, 29266), dtype=uint8, chunksize=(1024, 1024), chunktype=numpy.ndarray>

In [5]:
# import df
df = pd.read_pickle(df_path)

In [6]:
sam2 = build_sam2(model_cfg, sam2_checkpoint, device=device, apply_postprocessing=False)

mask_generator = SAM2AutomaticMaskGenerator(
    model=sam2,
    points_per_side=64,
    points_per_batch=64,
    pred_iou_thresh=0.7,
    stability_score_thresh=0.92,
    stability_score_offset=0.7,
    crop_n_layers=2,
    box_nms_thresh=0.7,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=1000,
    use_m2m=False,
)

In [7]:
res_adjust = axon_res - mitos_res
mito_props = ['area', 'predicted_iou', 'stability_score']

st = 110
for ind, row in tqdm(df.loc[st:, :].iterrows(), total=len(df.loc[st:, :])):

    # get the cell interior image of high res
    row_start = (int(row['inside_bbox-0']) + row_offset)*2**res_adjust
    row_end = (int(row['inside_bbox-2']) + row_offset)*2**res_adjust
    col_start = (int(row['inside_bbox-1']) + col_offset)*2**res_adjust
    col_end = (int(row['inside_bbox-3']) + col_offset)*2**res_adjust

    im_test = im[row_start:row_end, col_start:col_end]
    
    # run the segmentation
    im_rgb = gray2rgb(im_test).compute()
    masks_org = mask_generator.generate(im_rgb)

    # saving the masks
    file_name = f'{prefix_save}{str(row.label).zfill(6)}_mito.pkl'
    with open(os.path.join(output_dir, output_sub_dir, file_name), 'wb') as f:
        pkl.dump(masks_org, f)

    torch.cuda.empty_cache()
    del im_test
    del im_rgb 
    del masks_org
    gc.collect()

  0%|          | 0/478 [00:00<?, ?it/s]

/home/kasia/sam/sam2/sam2/sam2_image_predictor.py:431: UserWarning: cannot import name '_C' from 'sam2' (/home/kasia/sam/sam2/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  masks = self._transforms.postprocess_masks(
 92%|█████████▏| 438/478 [1:40:26<09:10, 13.76s/it]
